## 感知机
感知机是根据输入实例的特征向量 x 对其进行二类分类的线性模型：
$$ f(x)=sign(w\cdot x+b) $$
感知机模型对应于输入空间（特征空间）中的分离超平面 $ w\cdot x+b=0 $.其中w是超平面的法向量，b是超平面的截距。

可见感知机是一种线性分类模型，属于判别模型。
***
## 感知机学习的假设
感知机学习的重要前提假设是训练数据集是线性可分的。
***
## 感知机学习策略
感知机学的策略是极小化损失函数。

损失函数的一个自然选择是误分类点的总数。但是，这样的损失函数不是参数 w, b的连续可导的函数，不易于优化。所以通常是选择误分类点到超平面 S 的总距离：
$$ L(w,b)=-\sum_{x_i\in M}y_i(w\cdot x_i+b) $$
学习的策略就是求得使 L(w,b) 为最小值的 w 和 b。其中 M 是误分类点的集合。
***
## 感知机学习的算法
感知机学习算法是基于随机梯度下降法的对损失函数的最优化算法，有原始形式和对偶形式，算法简单易于实现。
### 原始形式
$$ \min_{w,b}L(w,b)=-\sum_{x_i\in M}y_i(w\cdot x_i+b) $$
首先，任意选取一个超平面$ w_0, b_0 $,然后用梯度下降法不断地极小化目标函数。极小化的过程中不是一次使 M 中所有误分类点得梯度下降，而是一次随机选取一个误分类点，使其梯度下降。
$$ \nabla_wL(w,b)=-\sum_{x_i\in M}y_ix_i $$
$$ \nabla_bL(w,b)=-\sum_{x_i\in M}y_i $$

随机选取一个误分类点$ (x_i,y_i) $，对 w,b 进行更新：
$$ w\leftarrow w+\eta y_ix_i $$
$$ b\leftarrow b+\eta y_i $$
其中$ \eta(0<\eta\leq1) $是学习率。

### 对偶形式
对偶形式的基本想法是，将 w 和 b 表示为是咧 $ x_i $ 和标记 $ y_i $的线性组合的形式，通过求解其系数而得到 w 和 b。
$$ w\leftarrow w+\eta y_ix_i $$
$$ b\leftarrow b+\eta y_i $$
逐步修改 w,b，设修改 n 次，则 w,b 关于$ (x_i,y_i) $ 的增量分别是 $ \alpha_iy_ix_i $ 和 $ \alpha_iy_i $, 这里 $ \alpha_i=n_i\eta $。最后学习到的 w,b 可以分别表示为：
$$ w=\sum_{i=1}^{N}\alpha_iy_ix_i $$
$$ b=\sum_{i=1}^{N}\alpha_iy_i $$
这里， $ \alpha_i\geq0, i=1,2,...,N $,当 $ \eta=1 $时，表示第i个是实例点由于误分类而进行更新的次数，实例点更新次数越多，说明它距离分离超平面越近，也就越难区分，该点对学习结果的影响最大。

感知机模型对偶形式： $$f(x)=sign(\sum_{j=1}^{N}\alpha_jy_jx_j\cdot x+b) $$ 其中$$\alpha=(\alpha_1,\alpha_2,...,\alpha_N)^T$$
学习时初始化 $ \alpha \leftarrow 0, b \leftarrow 0 $, 在训练集中找分类错误的点，即：
 $$ y_i(\sum_{j=1}^{N}\alpha_jy_jx_j\cdot x_i+b)\leq 0 $$
 然后更新：
 $$ \alpha_i \leftarrow \alpha_i+\eta$$
$$ b\leftarrow b+\eta y_i $$
知道训练集中所有点正确分类

对偶形式中训练实例仅以内积的形式出现，为了方便，可以预先将训练集中实例间的内积计算出来以矩阵的形式存储，即 Gram 矩阵。
***
## 总结

* 当训练数据集线性可分的时候，感知机学习算法是收敛的，感知机算法在训练数据集上的误分类次数 k 满足不等式:
$$ k\leq (\frac{R}{\gamma})^2  $$
具体证明可见 <font color=blue>李航《统计学习方法》或 林轩田《机器学习基石》</font>。

* 当训练当训练数据集线性可分的时候，感知机学习算法存在无穷多个解，其解由于不同的初值或不同的迭代顺序而可能不同，即存在多个分离超平面能把数据集分开。

* 感知机学习算法简单易求解，但一般的感知机算法不能解决异或等线性不可分的问题。

***
## 比较两个模型
分别从原始模型和对偶模型中获取参数，可以看出，这两个模型的分离超平面都不同，但是都能正确进行分类，这验证了总结中的结论。

当训练当训练数据集线性可分的时候，感知机学习算法存在无穷多个解，其解由于不同的初值或不同的迭代顺序而可能不同，即存在多个分离超平面能把数据集分开。
